In [1]:
import subprocess
import os
import statistics
from Bio import SeqIO, AlignIO
from Bio.Align import PairwiseAligner
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import tempfile

class SequenceAligner:
    def __init__(self, input_fasta, output_dir="/Users/sarawut/Desktop/oneclick/"):
        self.input_fasta = input_fasta
        self.output_dir = output_dir
        self.sequences = []
        self.filtered_sequences = []
        self.target_species = []
        
        # ตั้งค่าเกณฑ์การกรอง
        self.length_tolerance = 0.20  # 20%
        self.min_alignment_score = 0.6  # คะแนน alignment ขั้นต่ำ
        self.max_gap_percentage = 0.3   # % gaps สูงสุดที่ยอมรับได้
        
        # อ่านซีเควนซ์
        self._load_sequences()
    
    def _load_sequences(self):
        """อ่านซีเควนซ์จากไฟล์ FASTA"""
        print("🔍 กำลังอ่านซีเควนซ์...")
        self.sequences = list(SeqIO.parse(self.input_fasta, "fasta"))
        print(f"  พบซีเควนซ์ทั้งหมด: {len(self.sequences)}")
        
        # แสดงข้อมูลสถิติเบื้องต้น
        lengths = [len(seq.seq) for seq in self.sequences]
        print(f"  ความยาวเฉลี่ย: {statistics.mean(lengths):,.0f} bp")
        print(f"  ความยาวกลาง: {statistics.median(lengths):,.0f} bp")
        print(f"  สั้นสุด: {min(lengths):,.0f} bp")
        print(f"  ยาวสุด: {max(lengths):,.0f} bp")
    
    def set_target_species(self, species_keywords):
        """ตั้งค่าสปีชีส์หลักสำหรับการ alignment"""
        self.target_species = species_keywords
        print(f"🎯 ตั้งค่าสปีชีส์หลัก: {species_keywords}")
    
    def check_sequence_quality(self, sequence):
        """ตรวจสอบคุณภาพของซีเควนซ์"""
        seq_str = str(sequence.seq).upper()
        
        # นับเบสที่ไม่ใช่ A,T,C,G
        valid_bases = set('ATCG')
        invalid_count = sum(1 for base in seq_str if base not in valid_bases)
        total_bases = len(seq_str)
        
        # คำนวณเปอร์เซ็นต์เบสที่ไม่ถูกต้อง
        invalid_percentage = (invalid_count / total_bases) * 100 if total_bases > 0 else 100
        
        # เกณฑ์คุณภาพ: ยอมรับเบสที่ไม่ถูกต้องไม่เกิน 5%
        is_high_quality = invalid_percentage <= 5.0
        
        return {
            'is_high_quality': is_high_quality,
            'invalid_percentage': invalid_percentage,
            'invalid_count': invalid_count,
            'total_bases': total_bases
        }
    
    def remove_duplicates(self, sequences):
        """ตัด duplicate sequences ออก"""
        print("\n🔍 กำลังตรวจสอบ duplicate sequences...")
        
        seen_sequences = {}
        unique_sequences = []
        duplicates = []
        
        for seq in sequences:
            seq_hash = str(seq.seq).upper()
            
            if seq_hash in seen_sequences:
                # พบ duplicate
                original_id = seen_sequences[seq_hash]
                duplicates.append((seq.id, original_id))
            else:
                # ซีเควนซ์ใหม่
                seen_sequences[seq_hash] = seq.id
                unique_sequences.append(seq)
        
        print(f"  ✅ ซีเควนซ์ที่ไม่ซ้ำ: {len(unique_sequences)}")
        print(f"  🔄 Duplicates ที่ตัดออก: {len(duplicates)}")
        
        if duplicates:
            print("  รายการ duplicates ที่ตัดออก:")
            for dup_id, original_id in duplicates[:5]:
                print(f"    - {dup_id} (ซ้ำกับ {original_id})")
            if len(duplicates) > 5:
                print(f"    ... และอีก {len(duplicates)-5} รายการ")
        
        return unique_sequences
    
    def filter_by_quality(self, sequences):
        """กรองซีเควนซ์ตามคุณภาพ"""
        print("\n🧬 กำลังกรองซีเควนซ์ตามคุณภาพ...")
        
        high_quality = []
        low_quality = []
        
        for seq in sequences:
            quality = self.check_sequence_quality(seq)
            
            if quality['is_high_quality']:
                high_quality.append(seq)
            else:
                low_quality.append((seq.id, quality))
        
        print(f"  ✅ คุณภาพดี: {len(high_quality)} ซีเควนซ์")
        print(f"  ❌ คุณภาพต่ำ: {len(low_quality)} ซีเควนซ์")
        
        if low_quality:
            print("  รายการคุณภาพต่ำที่ตัดออก:")
            for seq_id, quality in low_quality[:5]:
                print(f"    - {seq_id}: {quality['invalid_percentage']:.1f}% เบสไม่ถูกต้อง")
            if len(low_quality) > 5:
                print(f"    ... และอีก {len(low_quality)-5} ซีเควนซ์")
        
        return high_quality
    
    def filter_by_length(self):
        """กรองซีเควนซ์ตามความยาว พร้อมตัด duplicates และคุณภาพต่ำ"""
        print("\n" + "="*50)
        print("📏 เริ่มกระบวนการกรองซีเควนซ์")
        print("="*50)
        
        # แสดงสถิติเริ่มต้น
        lengths = [len(seq.seq) for seq in self.sequences]
        print(f"📊 สถิติเริ่มต้น:")
        print(f"  จำนวนซีเควนซ์: {len(self.sequences)}")
        print(f"  ความยาวเฉลี่ย: {statistics.mean(lengths):,.0f} bp")
        print(f"  ความยาวกลาง: {statistics.median(lengths):,.0f} bp")
        print(f"  ช่วงความยาว: {min(lengths):,.0f} - {max(lengths):,.0f} bp")
        
        # ขั้นที่ 1: ตัด duplicates
        step1_sequences = self.remove_duplicates(self.sequences)
        
        # ขั้นที่ 2: กรองตามคุณภาพ
        step2_sequences = self.filter_by_quality(step1_sequences)
        
        # ขั้นที่ 3: กรองตามความยาว
        print("\n📏 กรองตามความยาว...")
        lengths = [len(seq.seq) for seq in step2_sequences]
        median_length = statistics.median(lengths)
        
        # คำนวณช่วงที่ยอมรับได้
        min_acceptable = median_length * (1 - self.length_tolerance)
        max_acceptable = median_length * (1 + self.length_tolerance)
        
        print(f"  ความยาวกลาง: {median_length:,.0f} bp")
        print(f"  ช่วงที่ยอมรับ: {min_acceptable:,.0f} - {max_acceptable:,.0f} bp")
        
        # กรองซีเควนซ์
        final_sequences = []
        length_removed = []
        
        for seq in step2_sequences:
            seq_length = len(seq.seq)
            if min_acceptable <= seq_length <= max_acceptable:
                final_sequences.append(seq)
            else:
                length_removed.append((seq.id, seq_length))
        
        self.filtered_sequences = final_sequences
        
        print(f"  ✅ ผ่านการคัดเลือก: {len(final_sequences)} ซีเควนซ์")
        print(f"  ❌ ความยาวไม่เหมาะสม: {len(length_removed)} ซีเควนซ์")
        
        if length_removed:
            print("  รายการที่ตัดออกเนื่องจากความยาว:")
            for seq_id, length in length_removed[:5]:
                print(f"    - {seq_id}: {length:,} bp")
            if len(length_removed) > 5:
                print(f"    ... และอีก {len(length_removed)-5} ซีเควนซ์")
        
        # สรุปผลการกรอง
        total_removed = len(self.sequences) - len(final_sequences)
        print(f"\n📋 สรุปการกรอง:")
        print(f"  เริ่มต้น: {len(self.sequences)} ซีเควนซ์")
        print(f"  เหลือหลัง duplicate removal: {len(step1_sequences)}")
        print(f"  เหลือหลัง quality filter: {len(step2_sequences)}")
        print(f"  สุดท้าย: {len(final_sequences)} ซีเควนซ์")
        print(f"  ตัดออกรวม: {total_removed} ซีเควนซ์ ({(total_removed/len(self.sequences)*100):.1f}%)")
    
    def _get_species_from_description(self, description):
        """แยกชื่อสปีชีส์จาก description"""
        # พยายามแยกชื่อสปีชีส์จาก description
        parts = description.split()
        if len(parts) >= 2:
            return f"{parts[0]} {parts[1]}"
        return description
    
    def organize_sequences_by_priority(self):
        """จัดเรียงซีเควนซ์ตามความสำคัญ (species หลักก่อน)"""
        print("\n🎯 จัดเรียงลำดับความสำคัญ...")
        
        priority_sequences = []
        other_sequences = []
        
        for seq in self.filtered_sequences:
            species_name = self._get_species_from_description(seq.description)
            is_target = any(target in species_name.lower() for target in 
                          [t.lower() for t in self.target_species])
            
            if is_target:
                priority_sequences.append(seq)
            else:
                other_sequences.append(seq)
        
        # รวมกัน (species หลักก่อน)
        organized_sequences = priority_sequences + other_sequences
        
        print(f"  🎯 สปีชีส์หลัก: {len(priority_sequences)} ซีเควนซ์")
        print(f"  📊 สปีชีส์อื่น: {len(other_sequences)} ซีเควนซ์")
        
        return organized_sequences
    
    def _calculate_alignment_quality(self, seq1, seq2):
        """คำนวณคุณภาพของการ alignment ระหว่าง 2 ซีเควนซ์"""
        try:
            aligner = PairwiseAligner()
            aligner.match_score = 2
            aligner.mismatch_score = -1
            aligner.open_gap_score = -2
            aligner.extend_gap_score = -0.5
            
            alignments = aligner.align(str(seq1.seq), str(seq2.seq))
            best_alignment = alignments[0]
            
            # คำนวณคะแนนที่ปรับแล้ว (0-1)
            max_possible_score = min(len(seq1.seq), len(seq2.seq)) * 2
            normalized_score = best_alignment.score / max_possible_score
            
            return max(0, min(1, normalized_score))
            
        except Exception as e:
            print(f"    ⚠️ ไม่สามารถคำนวณคะแนนได้: {e}")
            return 0.0
    
    def progressive_alignment(self):
        """ทำ alignment แบบค่อยเป็นค่อยไป พร้อมตรวจสอบคุณภาพและรายงานทุกช่วง"""
        print("\n" + "="*50)
        print("🔄 เริ่มการ Progressive Alignment")
        print("="*50)
        
        organized_sequences = self.organize_sequences_by_priority()
        
        if len(organized_sequences) < 2:
            print("❌ ต้องมีซีเควนซ์อย่างน้อย 2 ตัวสำหรับการ alignment")
            return None
        
        accepted_sequences = []
        rejected_sequences = []
        alignment_stats = []
        
        # เริ่มต้นด้วยซีเควนซ์แรก (ควรเป็น target species)
        accepted_sequences.append(organized_sequences[0])
        print(f"🎯 เริ่มต้นด้วยซีเควนซ์ reference: {organized_sequences[0].id}")
        print(f"   Species: {self._get_species_from_description(organized_sequences[0].description)}")
        print(f"   ความยาว: {len(organized_sequences[0].seq):,} bp")
        
        # ตัวแปรสำหรับการรายงาน
        report_interval = max(1, len(organized_sequences) // 10)  # รายงานทุก 10%
        total_sequences = len(organized_sequences) - 1
        
        # ทดสอบซีเควนซ์ทีละตัว
        for i, candidate_seq in enumerate(organized_sequences[1:], 1):
            progress_percent = (i / total_sequences) * 100
            
            print(f"\n📝 [{i:3d}/{total_sequences}] ({progress_percent:5.1f}%) ทดสอบ: {candidate_seq.id}")
            print(f"    Species: {self._get_species_from_description(candidate_seq.description)}")
            print(f"    ความยาว: {len(candidate_seq.seq):,} bp")
            
            # เปรียบเทียบกับซีเควนซ์ที่ยอมรับแล้ว
            quality_scores = []
            comparison_details = []
            
            # เลือกซีเควนซ์ที่จะใช้เปรียบเทียบ (สูงสุด 5 ตัวล่าสุด)
            comparison_seqs = accepted_sequences[-5:] if len(accepted_sequences) > 5 else accepted_sequences
            
            for j, accepted_seq in enumerate(comparison_seqs):
                score = self._calculate_alignment_quality(candidate_seq, accepted_seq)
                quality_scores.append(score)
                comparison_details.append({
                    'reference': accepted_seq.id[:30] + "..." if len(accepted_seq.id) > 30 else accepted_seq.id,
                    'score': score
                })
                print(f"    vs {accepted_seq.id[:25]:<25}...: {score:.3f}")
            
            # ตัดสินใจยอมรับหรือปฏิเสธ
            avg_score = statistics.mean(quality_scores) if quality_scores else 0
            max_score = max(quality_scores) if quality_scores else 0
            min_score = min(quality_scores) if quality_scores else 0
            
            # เกณฑ์การตัดสินใจ (ใช้คะแนนเฉลี่ย)
            decision_reason = ""
            if avg_score >= self.min_alignment_score:
                accepted_sequences.append(candidate_seq)
                decision = "✅ ยอมรับ"
                decision_reason = f"คะแนนเฉลี่ย {avg_score:.3f} ≥ {self.min_alignment_score}"
            else:
                rejected_sequences.append((candidate_seq, avg_score, comparison_details))
                decision = "❌ ปฏิเสธ"
                decision_reason = f"คะแนนเฉลี่ย {avg_score:.3f} < {self.min_alignment_score}"
            
            print(f"    📊 คะแนนสถิติ: เฉลี่ย={avg_score:.3f}, สูงสุด={max_score:.3f}, ต่ำสุด={min_score:.3f}")
            print(f"    {decision}: {decision_reason}")
            
            # เก็บสถิติสำหรับการรายงาน
            alignment_stats.append({
                'sequence_id': candidate_seq.id,
                'avg_score': avg_score,
                'max_score': max_score,
                'min_score': min_score,
                'accepted': avg_score >= self.min_alignment_score,
                'comparisons': len(quality_scores)
            })
            
            # รายงานความคืบหน้าทุกช่วง
            if i % report_interval == 0 or i == total_sequences:
                self._print_progress_report(i, total_sequences, len(accepted_sequences), len(rejected_sequences))
        
        # สรุปผลการคัดเลือก
        print(f"\n" + "="*50)
        print("📊 สรุปผล Progressive Alignment")
        print("="*50)
        print(f"✅ ยอมรับ: {len(accepted_sequences)} ซีเควนซ์")
        print(f"❌ ปฏิเสธ: {len(rejected_sequences)} ซีเควนซ์")
        print(f"📈 อัตราการยอมรับ: {(len(accepted_sequences)/len(organized_sequences)*100):.1f}%")
        
        # แสดงสถิติคะแนน
        if alignment_stats:
            scores = [stat['avg_score'] for stat in alignment_stats]
            print(f"\n📊 สถิติคะแนน alignment:")
            print(f"  คะแนนเฉลี่ย: {statistics.mean(scores):.3f}")
            print(f"  คะแนนกลาง: {statistics.median(scores):.3f}")
            print(f"  สูงสุด: {max(scores):.3f}")
            print(f"  ต่ำสุด: {min(scores):.3f}")
        
        # แสดงรายละเอียดซีเควนซ์ที่ปฏิเสธ
        if rejected_sequences:
            print(f"\n❌ รายละเอียดซีเควนซ์ที่ปฏิเสธ:")
            for seq, score, details in rejected_sequences[:10]:  # แสดงแค่ 10 อันแรก
                species = self._get_species_from_description(seq.description)
                print(f"  - {seq.id[:40]:<40} (Score: {score:.3f}) - {species}")
            if len(rejected_sequences) > 10:
                print(f"  ... และอีก {len(rejected_sequences)-10} ซีเควนซ์")
        
        # บันทึกซีเควนซ์ที่ผ่านการคัดเลือก
        filtered_file = os.path.join(self.output_dir, "filtered_sequences.fasta")
        SeqIO.write(accepted_sequences, filtered_file, "fasta")
        print(f"\n💾 บันทึกซีเควนซ์ที่คัดเลือกแล้ว: {filtered_file}")
        
        return accepted_sequences, filtered_file
    
    def _print_progress_report(self, current, total, accepted, rejected):
        """พิมพ์รายงานความคืบหน้า"""
        progress = (current / total) * 100
        print(f"\n📈 รายงานความคืบหน้า ({progress:.1f}%):")
        print(f"    ทดสอบแล้ว: {current}/{total} ซีเควนซ์")
        print(f"    ยอมรับ: {accepted} ซีเควนซ์")
        print(f"    ปฏิเสธ: {rejected} ซีเควนซ์")
        print(f"    อัตราการยอมรับ: {(accepted/(accepted+rejected)*100):.1f}%")
    
    def run_mafft_alignment(self, input_file, output_file):
        """รัน MAFFT alignment"""
        print(f"\n🔧 รัน MAFFT alignment...")
        try:
            # ใช้พารามิเตอร์ที่เหมาะสมสำหรับซีเควนซ์ที่คล้ายกัน
            result = subprocess.run([
                'mafft',
                '--adjustdirection',    # ปรับทิศทาง
                '--maxiterate', '1000', # จำนวนรอบสูงสุด
                '--globalpair',         # global pairwise alignment
                '--thread', '4',        # ใช้ 4 threads
                input_file
            ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, check=True)
            
            # เขียนผลลัพธ์
            with open(output_file, 'w') as f:
                f.write(result.stdout)
            
            print(f"  ✅ MAFFT alignment เสร็จสิ้น: {output_file}")
            return output_file
            
        except subprocess.CalledProcessError as e:
            print(f"  ❌ MAFFT ล้มเหลว: {e.stderr}")
            return None
    
    def analyze_alignment_quality(self, alignment_file):
        """วิเคราะห์คุณภาพของ alignment อย่างละเอียด"""
        print(f"\n" + "="*50)
        print("📈 วิเคราะห์คุณภาพ Alignment")
        print("="*50)
        
        try:
            alignment = AlignIO.read(alignment_file, 'fasta')
            
            # ข้อมูลพื้นฐาน
            alignment_length = alignment.get_alignment_length()
            num_sequences = len(alignment)
            
            print(f"📊 ข้อมูลพื้นฐาน:")
            print(f"  📏 ความยาว alignment: {alignment_length:,} ตำแหน่ง")
            print(f"  🧬 จำนวนซีเควนซ์: {num_sequences}")
            
            # วิเคราะห์ gaps
            print(f"\n🕳️  การวิเคราะห์ Gaps:")
            gap_stats = self._analyze_gaps(alignment)
            
            print(f"  รวม gaps ทั้งหมด: {gap_stats['total_gaps']:,} ({gap_stats['gap_percentage']:.1f}%)")
            print(f"  เฉลี่ยต่อซีเควนซ์: {gap_stats['avg_gaps_per_seq']:.1f} gaps")
            print(f"  ซีเควนซ์ที่มี gaps มากสุด: {gap_stats['max_gaps_seq']:.1f}%")
            print(f"  ซีเควนซ์ที่มี gaps น้อยสุด: {gap_stats['min_gaps_seq']:.1f}%")
            
            # วิเคราะห์ conservation
            print(f"\n🔬 การวิเคราะห์ Conservation:")
            conservation_stats = self._analyze_conservation(alignment)
            
            print(f"  ตำแหน่งที่ conserved 100%: {conservation_stats['fully_conserved']:,} ({conservation_stats['fully_conserved_percent']:.1f}%)")
            print(f"  ตำแหน่งที่ conserved ≥90%: {conservation_stats['highly_conserved']:,} ({conservation_stats['highly_conserved_percent']:.1f}%)")
            print(f"  ตำแหน่งที่ conserved ≥70%: {conservation_stats['moderately_conserved']:,} ({conservation_stats['moderately_conserved_percent']:.1f}%)")
            print(f"  ตำแหน่งที่ variable: {conservation_stats['variable']:,} ({conservation_stats['variable_percent']:.1f}%)")
            
            # วิเคราะห์ซีเควนซ์แต่ละตัว
            print(f"\n🧬 การวิเคราะห์ซีเควนซ์แต่ละตัว:")
            seq_stats = self._analyze_individual_sequences(alignment)
            
            # แสดงซีเควนซ์ที่มีคุณภาพดีที่สุดและแย่ที่สุด
            best_seq = min(seq_stats, key=lambda x: x['gap_percent'])
            worst_seq = max(seq_stats, key=lambda x: x['gap_percent'])
            
            print(f"  ✅ คุณภาพดีที่สุด: {best_seq['id'][:50]} ({best_seq['gap_percent']:.1f}% gaps)")
            print(f"  ⚠️  คุณภาพแย่ที่สุด: {worst_seq['id'][:50]} ({worst_seq['gap_percent']:.1f}% gaps)")
            
            # ประเมินคุณภาพโดยรวม
            print(f"\n🎯 ประเมินคุณภาพโดยรวม:")
            overall_quality = self._evaluate_overall_quality(gap_stats, conservation_stats)
            
            print(f"  คะแนนคุณภาพ: {overall_quality['score']:.2f}/10")
            print(f"  ระดับคุณภาพ: {overall_quality['grade']}")
            print(f"  คำแนะนำ: {overall_quality['recommendation']}")
            
            # สรุปสถิติสำคัญ
            summary_stats = {
                'length': alignment_length,
                'sequences': num_sequences,
                'gap_percentage': gap_stats['gap_percentage'],
                'conservation_score': conservation_stats['conservation_score'],
                'quality_score': overall_quality['score'],
                'quality_grade': overall_quality['grade']
            }
            
            return summary_stats
            
        except Exception as e:
            print(f"  ❌ ไม่สามารถวิเคราะห์ได้: {e}")
            return None
    
    def _analyze_gaps(self, alignment):
        """วิเคราะห์ gaps ในรายละเอียด"""
        total_gaps = 0
        total_bases = 0
        seq_gap_percents = []
        
        for record in alignment:
            sequence = str(record.seq)
            gaps = sequence.count('-')
            total_gaps += gaps
            total_bases += len(sequence)
            seq_gap_percent = (gaps / len(sequence)) * 100
            seq_gap_percents.append(seq_gap_percent)
        
        return {
            'total_gaps': total_gaps,
            'total_bases': total_bases,
            'gap_percentage': (total_gaps / total_bases) * 100,
            'avg_gaps_per_seq': statistics.mean(seq_gap_percents),
            'max_gaps_seq': max(seq_gap_percents),
            'min_gaps_seq': min(seq_gap_percents)
        }
    
    def _analyze_conservation(self, alignment):
        """วิเคราะห์ conservation ในแต่ละตำแหน่ง"""
        alignment_length = alignment.get_alignment_length()
        num_sequences = len(alignment)
        
        fully_conserved = 0
        highly_conserved = 0
        moderately_conserved = 0
        variable = 0
        conservation_scores = []
        
        for pos in range(alignment_length):
            # ดึงเบสในตำแหน่งนี้จากทุกซีเควนซ์
            bases = [alignment[i].seq[pos].upper() for i in range(num_sequences)]
            
            # นับเบสที่ไม่ใช่ gap
            non_gap_bases = [base for base in bases if base != '-']
            
            if len(non_gap_bases) == 0:
                conservation_scores.append(0)
                variable += 1
                continue
            
            # นับเบสที่เหมือนกันมากที่สุด
            from collections import Counter
            base_counts = Counter(non_gap_bases)
            most_common_count = base_counts.most_common(1)[0][1]
            
            # คำนวณเปอร์เซ็นต์ conservation
            conservation_percent = (most_common_count / len(non_gap_bases)) * 100
            conservation_scores.append(conservation_percent)
            
            # จัดหมวดหมู่
            if conservation_percent == 100:
                fully_conserved += 1
            elif conservation_percent >= 90:
                highly_conserved += 1
            elif conservation_percent >= 70:
                moderately_conserved += 1
            else:
                variable += 1
        
        return {
            'fully_conserved': fully_conserved,
            'fully_conserved_percent': (fully_conserved / alignment_length) * 100,
            'highly_conserved': highly_conserved,
            'highly_conserved_percent': (highly_conserved / alignment_length) * 100,
            'moderately_conserved': moderately_conserved,
            'moderately_conserved_percent': (moderately_conserved / alignment_length) * 100,
            'variable': variable,
            'variable_percent': (variable / alignment_length) * 100,
            'conservation_score': statistics.mean(conservation_scores)
        }
    
    def _analyze_individual_sequences(self, alignment):
        """วิเคราะห์ซีเควนซ์แต่ละตัว"""
        seq_stats = []
        
        for record in alignment:
            sequence = str(record.seq)
            gaps = sequence.count('-')
            gap_percent = (gaps / len(sequence)) * 100
            
            seq_stats.append({
                'id': record.id,
                'length': len(sequence),
                'gaps': gaps,
                'gap_percent': gap_percent,
                'effective_length': len(sequence) - gaps
            })
        
        return seq_stats
    
    def _evaluate_overall_quality(self, gap_stats, conservation_stats):
        """ประเมินคุณภาพโดยรวม"""
        score = 0
        
        # คะแนนจาก gap percentage (น้อยกว่า = ดีกว่า)
        if gap_stats['gap_percentage'] < 10:
            score += 3
        elif gap_stats['gap_percentage'] < 20:
            score += 2
        elif gap_stats['gap_percentage'] < 40:
            score += 1
        
        # คะแนนจาก conservation
        if conservation_stats['conservation_score'] > 80:
            score += 3
        elif conservation_stats['conservation_score'] > 60:
            score += 2
        elif conservation_stats['conservation_score'] > 40:
            score += 1
        
        # คะแนนจาก fully conserved positions
        if conservation_stats['fully_conserved_percent'] > 30:
            score += 2
        elif conservation_stats['fully_conserved_percent'] > 15:
            score += 1
        
        # คะแนนจากความสม่ำเสมอของ gaps
        gap_range = gap_stats['max_gaps_seq'] - gap_stats['min_gaps_seq']
        if gap_range < 10:
            score += 2
        elif gap_range < 25:
            score += 1
        
        # กำหนดเกรดและคำแนะนำ
        if score >= 8:
            grade = "เยี่ยม (A)"
            recommendation = "alignment มีคุณภาพสูงมาก เหมาะสำหรับการออกแบบไพรเมอร์"
        elif score >= 6:
            grade = "ดี (B)"
            recommendation = "alignment มีคุณภาพดี สามารถใช้ออกแบบไพรเมอร์ได้"
        elif score >= 4:
            grade = "พอใช้ (C)"
            recommendation = "alignment มีคุณภาพปานกลาง ควรพิจารณากรองซีเควนซ์เพิ่มเติม"
        else:
            grade = "ต้องปรับปรุง (D)"
            recommendation = "alignment มีคุณภาพต่ำ แนะนำให้กรองซีเควนซ์ใหม่หรือปรับพารามิเตอร์"
        
        return {
            'score': score,
            'grade': grade,
            'recommendation': recommendation
        }
    
    def run_complete_pipeline(self, target_species=None):
        """รันกระบวนการทั้งหมดแบบครบวงจร"""
        print("🚀 เริ่มต้นกระบวนการ High-Quality Alignment Pipeline")
        print("="*60)
        print(f"📅 เวลาเริ่มต้น: {time.strftime('%Y-%m-%d %H:%M:%S')}")
        
        start_time = time.time()
        
        # ตั้งค่า target species
        if target_species:
            self.set_target_species(target_species)
        
        # ขั้นที่ 1: กรองซีเควนซ์แบบครอบคลุม
        print(f"\n🔥 ขั้นที่ 1: การกรองซีเควนซ์แบบครอบคลุม")
        self.filter_by_length()
        
        if len(self.filtered_sequences) < 2:
            print("❌ ซีเควนซ์ไม่เพียงพอสำหรับ alignment หลังการกรอง")
            return None
        
        # ขั้นที่ 2: Progressive alignment พร้อมควบคุมคุณภาพ
        print(f"\n🔥 ขั้นที่ 2: Progressive Alignment")
        result = self.progressive_alignment()
        if not result:
            print("❌ Progressive alignment ล้มเหลว")
            return None
        
        accepted_sequences, filtered_file = result
        
        if len(accepted_sequences) < 2:
            print("❌ ซีเควนซ์ไม่เพียงพอสำหรับ MAFFT alignment")
            return None
        
        # ขั้นที่ 3: รัน MAFFT alignment
        print(f"\n🔥 ขั้นที่ 3: MAFFT Alignment")
        final_alignment = os.path.join(self.output_dir, "final_alignment_ON_COI.fasta")
        alignment_file = self.run_mafft_alignment(filtered_file, final_alignment)
        
        if not alignment_file:
            print("❌ MAFFT alignment ล้มเหลว")
            return None
        
        # ขั้นที่ 4: วิเคราะห์คุณภาพแบบละเอียด
        print(f"\n🔥 ขั้นที่ 4: การวิเคราะห์คุณภาพ")
        quality_stats = self.analyze_alignment_quality(alignment_file)
        
        if not quality_stats:
            print("⚠️ ไม่สามารถวิเคราะห์คุณภาพได้ แต่ alignment สำเร็จแล้ว")
        
        # คำนวณเวลาที่ใช้
        end_time = time.time()
        elapsed_time = end_time - start_time
        
        # สรุปผลลัพธ์สุดท้าย
        print("\n" + "="*60)
        print("🎉 กระบวนการเสร็จสิ้นสมบูรณ์!")
        print("="*60)
        
        print(f"📁 ไฟล์ผลลัพธ์:")
        print(f"  🧬 Alignment สุดท้าย: {alignment_file}")
        print(f"  📊 ซีเควนซ์ที่คัดเลือก: {filtered_file}")
        
        print(f"\n📊 สถิติสรุป:")
        print(f"  ⏱️  เวลาที่ใช้: {elapsed_time:.1f} วินาที")
        print(f"  📝 ซีเควนซ์เริ่มต้น: {len(self.sequences)}")
        print(f"  ✅ ซีเควนซ์สุดท้าย: {len(accepted_sequences)}")
        print(f"  🎯 อัตราการคัดเลือก: {(len(accepted_sequences)/len(self.sequences)*100):.1f}%")
        
        if quality_stats:
            print(f"  📏 ความยาว alignment: {quality_stats['length']:,} bp")
            print(f"  🕳️  Gaps: {quality_stats['gap_percentage']:.1f}%")
            print(f"  🏆 คะแนนคุณภาพ: {quality_stats['quality_score']:.1f}/10 ({quality_stats['quality_grade']})")
        
        print(f"\n💡 สรุปความพร้อมสำหรับการออกแบบไพรเมอร์:")
        
        if quality_stats and quality_stats['quality_score'] >= 6:
            print("  ✅ Alignment มีคุณภาพดี พร้อมสำหรับการออกแบบไพรเมอร์")
            print("  🎯 แนะนำให้มองหาบริเวณที่ conserved สูงสำหรับไพรเมอร์")
            print("  🔬 ใช้บริเวณ variable สำหรับการแยกแยะสปีชีส์")
        elif quality_stats and quality_stats['quality_score'] >= 4:
            print("  ⚠️ Alignment มีคุณภาพปานกลาง")
            print("  💡 แนะนำให้ตรวจสอบบริเวณที่มี gaps มากก่อนออกแบบไพรเมอร์")
        else:
            print("  ❌ Alignment อาจต้องการการปรับปรุง")
            print("  🔧 พิจารณาปรับเกณฑ์การกรองหรือใช้พารามิเตอร์ MAFFT อื่น")
        
        return alignment_file, quality_stats

# ===== ฟังก์ชันเสริม =====
import time

def print_welcome_banner():
    """แสดงแบนเนอร์ต้อนรับ"""
    print("="*70)
    print("🧬 HIGH-QUALITY SEQUENCE ALIGNMENT PIPELINE 🧬")
    print("="*70)
    print("🎯 จุดประสงค์: เตรียมซีเควนซ์สำหรับการออกแบบไพรเมอร์และโพรบ")
    print("⚡ คุณสมบัติ: กรองคุณภาพ, ตัด duplicates, progressive alignment")
    print("🔬 ผลลัพธ์: Alignment คุณภาพสูงพร้อมการวิเคราะห์ละเอียด")
    print("="*70)

# ===== การใช้งาน =====
if __name__ == "__main__":
    # แสดงแบนเนอร์ต้อนรับ
    print_welcome_banner()
    
    # ตั้งค่าไฟล์อินพุต
    input_fasta = "/Users/sarawut/Desktop/oneclick/thai_fish_sequences_ON_COI.fasta"
    
    # ตรวจสอบไฟล์อินพุต
    if not os.path.exists(input_fasta):
        print(f"❌ ไม่พบไฟล์อินพุต: {input_fasta}")
        print("💡 กรุณาตรวจสอบเส้นทางไฟล์และลองใหม่")
        exit(1)
    
    # สร้าง aligner object
    try:
        aligner = SequenceAligner(input_fasta)
    except Exception as e:
        print(f"❌ ไม่สามารถสร้าง aligner ได้: {e}")
        exit(1)
    
    # ตั้งค่าสปีชีส์หลัก (ปรับตามความต้องการ)
    target_species = [
        "oreochromis niloticus",  # สปีชีส์หลักที่ต้องการ
        # หรือใช้คำสำคัญในชื่อสปีชีส์
        # เพิ่มสปีชีส์หลักอื่นๆ ได้ที่นี่
    ]
    
    print(f"\n🎯 การตั้งค่าสำหรับการรัน:")
    print(f"  📁 ไฟล์อินพุต: {input_fasta}")
    print(f"  🎯 สปีชีส์หลัก: {target_species}")
    print(f"  📊 เกณฑ์ความยาว: ±{aligner.length_tolerance*100:.0f}% จากค่ากลาง")
    print(f"  🔬 คะแนน alignment ขั้นต่ำ: {aligner.min_alignment_score}")
    print(f"  🕳️  % gaps สูงสุดที่ยอมรับ: {aligner.max_gap_percentage*100:.0f}%")
    
    # ถามผู้ใช้ว่าจะดำเนินการต่อหรือไม่
    print(f"\n❓ ต้องการดำเนินการต่อหรือไม่? (y/n): ", end="")
    
    # สำหรับการรันแบบอัตโนมัติ สามารถข้ามส่วนนี้ได้
    # user_confirm = input().lower().strip()
    # if user_confirm not in ['y', 'yes', 'ใช่']:
    #     print("🚫 ยกเลิกการดำเนินการ")
    #     exit(0)
    
    print("y")  # สำหรับการรันอัตโนมัติ
    
    # รันกระบวนการทั้งหมด
    try:
        result = aligner.run_complete_pipeline(target_species)
        
        if result:
            alignment_file, stats = result
            print(f"\n✨ สำเร็จ!")
            print(f"📁 ไฟล์ alignment: {alignment_file}")
            
            if stats:
                print(f"🏆 คะแนนคุณภาพ: {stats['quality_score']:.1f}/10")
                
                # แนะนำขั้นตอนต่อไป
                print(f"\n🚀 ขั้นตอนต่อไป:")
                print(f"  1. ตรวจสอบไฟล์ alignment ด้วย sequence viewer")
                print(f"  2. ระบุบริเวณ conserved สำหรับการออกแบบไพรเมอร์")
                print(f"  3. หาบริเวณ variable สำหรับการแยกแยะสปีชีส์")
                print(f"  4. ใช้เครื่องมือออกแบบไพรเมอร์เช่น Primer3 หรือ NCBI Primer-BLAST")
            
        else:
            print("\n❌ กระบวนการล้มเหลว")
            print("💡 แนะนำ:")
            print("  - ตรวจสอบคุณภาพของซีเควนซ์อินพุต")
            print("  - ลองปรับเกณฑ์การกรองให้หลวมขึ้น")
            print("  - ตรวจสอบว่าซีเควนซ์มาจากสปีชีส์ที่เกี่ยวข้องกัน")
            
    except KeyboardInterrupt:
        print(f"\n⚠️ ผู้ใช้หยุดการทำงาน")
    except Exception as e:
        print(f"\n❌ เกิดข้อผิดพลาดไม่คาดคิด: {e}")
        print("💡 กรุณาตรวจสอบไฟล์อินพุตและลองใหม่")
    
    print(f"\n📅 เวลาสิ้นสุด: {time.strftime('%Y-%m-%d %H:%M:%S')}")
    print("🙏 ขอบคุณที่ใช้ High-Quality Sequence Alignment Pipeline!")

🧬 HIGH-QUALITY SEQUENCE ALIGNMENT PIPELINE 🧬
🎯 จุดประสงค์: เตรียมซีเควนซ์สำหรับการออกแบบไพรเมอร์และโพรบ
⚡ คุณสมบัติ: กรองคุณภาพ, ตัด duplicates, progressive alignment
🔬 ผลลัพธ์: Alignment คุณภาพสูงพร้อมการวิเคราะห์ละเอียด
🔍 กำลังอ่านซีเควนซ์...
  พบซีเควนซ์ทั้งหมด: 1642
  ความยาวเฉลี่ย: 2,693 bp
  ความยาวกลาง: 651 bp
  สั้นสุด: 140 bp
  ยาวสุด: 17,388 bp

🎯 การตั้งค่าสำหรับการรัน:
  📁 ไฟล์อินพุต: /Users/sarawut/Desktop/oneclick/thai_fish_sequences_ON_COI.fasta
  🎯 สปีชีส์หลัก: ['oreochromis niloticus']
  📊 เกณฑ์ความยาว: ±20% จากค่ากลาง
  🔬 คะแนน alignment ขั้นต่ำ: 0.6
  🕳️  % gaps สูงสุดที่ยอมรับ: 30%

❓ ต้องการดำเนินการต่อหรือไม่? (y/n): y
🚀 เริ่มต้นกระบวนการ High-Quality Alignment Pipeline
📅 เวลาเริ่มต้น: 2025-06-16 15:16:50
🎯 ตั้งค่าสปีชีส์หลัก: ['oreochromis niloticus']

🔥 ขั้นที่ 1: การกรองซีเควนซ์แบบครอบคลุม

📏 เริ่มกระบวนการกรองซีเควนซ์
📊 สถิติเริ่มต้น:
  จำนวนซีเควนซ์: 1642
  ความยาวเฉลี่ย: 2,693 bp
  ความยาวกลาง: 651 bp
  ช่วงความยาว: 140 - 17,388 bp

🔍 กำลังตรวจสอบ duplicate 